In [1]:
##### 同時推定コード #####
# ver1201 グラフの向き情報をなくしてみればリンクが少なくなるな．エスカレータ情報は一旦諦める
print('aiueo')
import pandas as pd 
import os
import datetime
import numpy as np
from datetime import timedelta 
import csv
import time
import math
from scipy.stats import norm
from scipy.optimize import minimize
import random

start_time = time.time()
print('start!')

aiueo
start!


In [2]:
# inputも時間間隔20secで実行
df_quater = pd.read_csv('/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/20230130_08/20230130_0845_20sec.csv')
search_folder = "/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/20230130_08/20230130_0845_20sec_end162021"
file_list = list(os.listdir(search_folder))

# df_linkも作り直し
df_link = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_stanw/stanw_link_post.csv')
L = len(df_link)

# df_nodeは作り直し
df_node = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_stanw/stanw_node.csv')

df_ble = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_nw_detail/ble_nw.csv')

In [41]:

###### 各linkのoとdの座標を入れておく配列 ###### # 線分とビーコンの距離に関しての情報は向きがあってもなくても変わらない
oddslink_loc_array = [] 
for i in range(len(df_link)):
    linkid = df_link.loc[i, 'linkid']
    O = df_link.loc[i, 'o']
    D = df_link.loc[i, 'd']

    x_o = df_node[df_node['nodeid'] == O]['x'].iloc[0]
    y_o = df_node[df_node['nodeid'] == O]['y'].iloc[0]
    z_o = df_node[df_node['nodeid'] == O]['floor'].iloc[0] 
    o_loc = np.array([x_o, y_o, z_o])

    x_d = df_node[df_node['nodeid'] == D]['x'].iloc[0]
    y_d = df_node[df_node['nodeid'] == D]['y'].iloc[0]
    z_d = df_node[df_node['nodeid'] == D]['floor'].iloc[0] # これでseriesではなく値として抽出できるはず
    d_loc = np.array([x_d, y_d, z_d])

    loc_tuple = (o_loc, d_loc)

    oddslink_loc_array.append(loc_tuple) # oddslink_loc_array[linkid//2][0]がlinkidのonode, [1]がdnode，[2]が中点，，，，

###### ビーコンとリンク線分の中点間距離を返す関数 ######
def shortest_distance_to_segment(p1, p2, x):
    p1 = np.array(p1)
    p2 = np.array(p2)
    x = np.array(x)
    
    segment = p2 - p1
    v1 = x - p1
    v2 = x - p2
    mid_point = (p1 + p2) / 2

    # 垂直ベクトルを計算
    v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
    distance = np.linalg.norm(x - mid_point)
    
    return distance

###### d_arrayの用意 ###### # 各リンク(の中点）とbleビーコンとの距離配列
d_array = np.zeros((len(df_link), len(df_ble)))

for i in range(len(df_link)):
    p_o = oddslink_loc_array[i][0] # o座標 # i=1の時
    p_d = oddslink_loc_array[i][1] # d座標

    for j in range(len(df_ble)):
        x_ap = df_ble.loc[j, 'x']
        y_ap = df_ble.loc[j, 'y']
        z_ap = df_ble.loc[j, 'floor']
        p_ap = np.array([x_ap, y_ap, z_ap])

        d_array[i, j] = shortest_distance_to_segment(p_o, p_d, p_ap) # ijはlink i+1, ビーコンj+1の距離

###### 混雑度dens，maxRSSI，meanRSSI，検出回数を各ビーコン各timestepごとに入れた配列（ビーコン数23*タイムステップ数） ######
### 20秒にするのでここやり直す．
dens_jt = np.zeros((23, 45), dtype = int)
max_jt = np.zeros((23, 45), dtype = int)
mean_jt = np.zeros((23, 45), dtype = int)
#count_jt = np.zeros((23, 45), dtype = int)

grouped = df_quater.groupby(['ID', 'timestep']) # listにして渡す必要がある
df_list = [group.reset_index(drop=True) for name, group in grouped]

for group in df_list:
    # beacon_idとtimestepを得る
    beacon_id = int(group.loc[0, 'ID'])
    timestep = int(group.loc[0, 'timestep'])

    # ユニークMACアドレス数，最大&平均RSSI
    unique_mac_count = group['MAC'].nunique()
    max_rssi = group['RSSI'].max()
    mean_rssi = group['RSSI'].mean()
    # countはそのタイムステップでの全検出数つまりgroupの長さそのもの
    #count = len(group)

    # numpy配列の要素はシーケンスダ
    dens_jt[beacon_id-1, timestep-1] = unique_mac_count
    max_jt[beacon_id-1, timestep-1] = max_rssi
    mean_jt[beacon_id-1, timestep-1] = mean_rssi
    #count_jt[beacon_id-1, timestep-1] = count

####### リンク接続行列（無向） ####### 
def I_nod(link_data): 
    n = len(link_data)
    I = np.eye(n) # 滞在リンク（吸収リンクにもつけて良い）
    for i in range(n):
        O = link_data.loc[i, 'o'] # 当該linkのOnode
        D = link_data.loc[i, 'd'] # 当該linkのDnode
        for j in range(n): # リンクiのOorDをOないしDとしているリンクjとは接続
            if ((link_data.loc[j, 'o'] == O) or (link_data.loc[j, 'o'] == D)) or (link_data.loc[j, 'd'] == O) or (link_data.loc[j, 'd'] == D): 
                I[i, j] = 1
    return(I)

I = I_nod(df_link)

###### 階段リンクについて別途処理 ###### # i = 26~44
for i in range(26, 45, 2):
    # 階段には滞在リンク入れない．向きは考えないので面倒なことは考えない
    I[i-1, i-1] = 0
    I[i, i] = 0

###### 吸収リンクからは吸収リンクにしか接続しない
for i in range(46, 49):
    I[i-1, :] = 0 # 他には繋がらない
    # I[i-1, i-1] = 1 # 自分自身にはつながる


###### 改札の吸収リンク
# ddata = [91, 92, 93]
# ddata = [33, 35, 49]
ddata = [46, 47, 48]
# ddata = []

D = len(ddata)

####### パラメタ初期設定 #######
xdim = 1
x0 = np.zeros(xdim)
x0[0] = -1
# x0[-1] = 0.8




####### 観測方程式 ###### （観測確率．リンクkにいるとき，最大スコアのビーコンがjになる確率）
def mq(dist, distmin):  # 基本的にdistに反比例するか，それか何かの分布を与える．とりあえず試すしかないわ．
    if dist >= 300: # distが25超えたら0にする
        return 0
    else:
        return (distmin / dist) 

####### 観測方程式（ビーコンの評価） ######
def beacon_eval(rssi, max_rssi):  # , freq): # 同時刻内で捕捉されたビーコンを評価．最大rssiと観測頻度による．20secも幅があるので
    return (max_rssi/rssi) # + freq) # 同一のビーコンからの観測強度の逆数を積み上げる

####### 観測モデル ###### # 各個人の各tiemstepに対して最もスコアの高いビーコンを決めるので出力はuserid, timestep, maxbeacon, absorption
def measuring_model(x):

    print(f'paramaterは{x}')
    qall = np.zeros(4)
    
    for file_name in file_list:
        if file_name.endswith('.csv'):
            base_name = file_name.split('.')[0]   
            userid = int(base_name)
                    
            file_path = os.path.join(search_folder, file_name)
            dfi = pd.read_csv(file_path) 
            nts = dfi['timestep'].nunique()                

            grouped = dfi.groupby('timestep')
            dfi_list = [group.reset_index(drop=True) for name, group in grouped]  
            qi = np.zeros(4)  # userid, timestep, maxbeacon, absorptionを後で入れる

            for t in range(nts):
                dfit = dfi_list[t]
                ts = int(dfit.loc[0, 'timestep'])

                count_tot = len(dfit) # 個人i時点tのcountの総和はdfitの長さそのもの
                        
                result_beacon = dfit.groupby('ID')
                result_beacon_list = [group.reset_index(drop=True) for name, group in result_beacon]

                # 各ビーコンのscoreを記録するためのdict
                dict = {}

                for j in range(len(result_beacon_list)): # 各ビーコンの結果を束ねたdataframe
                    dfitj = result_beacon_list[j]
                    jscore = 0 # 初期値は0にしておく
                    # count = len(dfitj) # そのビーコンでの検出回数→freqはやっぱりアレなので強度を全て足すようにする
                    # rssi = dfitj['RSSI'].max() # そのビーコンでの最大観測強度
                    jid = dfitj.loc[0, 'ID'] # このビーコンのid
                    max_rssi = max_jt[jid-1, ts-1] # tsでのbeaconのmaxのrssi

                    for k in range(len(dfitj)):
                        # freq = count / count_tot
                        rssi = dfitj.loc[k, 'RSSI']                        
                        # このビーコンのスコアを計算
                        jscore += beacon_eval(rssi, max_rssi) # , freq)

                    # dictにjidとjscoreを入れる
                    dict[jid] = jscore

                # dictに全部入っているので比較
                max_beacon = max(dict, key = dict.get) # score maxのビーコン

                # 個人i時刻tの結果（最もスコアの高かったビーコン）．absorptionの列はとりあえず0のまま残しておく
                qit = np.array([userid, ts, max_beacon, 0])
                qi = np.vstack((qi, qit))
            
            qi = np.delete(qi, 0, axis = 0)

            # ここまでで終了したので，qiの最後の行のmaxbeaconの値を取得しそれを最後の列に入れる
            abs = qi[-1, 2] # これで最後の行のmaxbeaconの列（0から数えて2列目）
            
            # qiの最後の列の値を全てabsにする
            qi[:, -1] = abs

            # 全個人に対してvstackしていってqallを作る
            qall = np.vstack((qall, qi))

    qall = np.delete(qall, 0, axis = 0)
    return qall 




###### 経路選択モデル ####### 
###### 初期設定 ######
V0 = np.full((L, D), -1) # exp(0)=1よりexp(-1)は小さくなるから?
z0 = np.exp(V0)

###### TSを設定しないといけない ###### #### 一斉に発生すると考えて良さそう→とりあえず時間構造化しないで解く
#TS = 10

V = V0
z = z0
# beta = x0[0] # dRLなのでbeta推定．xの最後の要素をbetaとする
# beta = 0.9 # dRLなのでbeta推定．xの最後の要素をbetaとする

###### 即時効用行列 ######
def Mset(x): 
    cost = np.exp(df_link['length'] * x[0]) # + df_link['esc'] * x[1]) # + df_link_integrated['staire_with_esc'] * x[3])
    cost = pd.concat([cost]*L, axis=1)
    cost = cost.T
    cost_numpy = cost.values # DataFrameをNumPy配列に変換
    return cost_numpy

###### 価値関数 ###### # 時間構造化するなら変わってくるが，，，
def newV(x):
    V = np.zeros((L, D))
    z = np.exp(V)

    beta = x[-1]
    
    for d in range(D): 
        d_linkid = ddata[d]
        d_index = df_link[df_link['linkid'] == d_linkid].index 
        z[d_index, d] = 1                   # 目的地がdの時，dでの期待効用Vd(d)=0から
        M = np.zeros((L, L))
        B = np.zeros((L, 1))
        B[d_index, 0] = 1
        # print(f'Iは{np.diag(I)}')

        # Mをdごとに更新
        for k in range(L): 
            for a in range(L): 
                Ika = I[k, a] 
                if Ika == 1:                # 接続してなければ無関係（0のまま）
                    if a == d_index: 
                        M[k, a] = 1         # 吸収リンクの即時効用はexp(0)=1
                    else:
                        M[k, a] = Mset(x)[k, a]
        dL = 100
        zd = z[:, d].reshape(L, 1) 
        #print(f'Mは{np.diag(M)}')
        # z求解
        count = 0
        while dL >= 0.01:
            zdd = zd.copy()
            zd = M @ (zdd ** beta) + B 
            dL = np.sum(np.abs(zdd - zd)) 
            # dL = np.linalg.norm(zdd - zd, axis=0) 
            count += 1

        # z更新
        z[:, d] = zd[:, 0] 
        zd = np.where(zd == 0, 1, zd)  
        one_dim_array = np.log(zd).ravel()   # 1次元配列に
        V[:, d] = one_dim_array 

    return z

###### 対数尤度関数 ###### 
def loglikelihood(x):
    LL = 0
    beta = x[-1]

    # 価値関数更新
    Z = newV(x) 

    print(f'パラメタ{x}で観測モデルを回します')
    qall = measuring_model(x) # 観測モデルの結果

    # 結果のうち，absorption（最後列）が16, 20, 21のいずれでもない場合消去
    condition = np.logical_or.reduce((qall[:, -1] == 16, qall[:, -1] == 20, qall[:, -1] == 21))
    indices_to_keep = np.where(condition)[0]
    filtered_qall = qall[indices_to_keep]
    mres = pd.DataFrame(filtered_qall, columns=['userid', 'timestep', 'maxbeacon', 'absorption'])
    
    # 観測結果をdで分割
    grouped = mres.groupby('absorption')
    df_list = [group for name, group in grouped]
    df_list = [data.reset_index(drop=True) for data in df_list]    


    for d in range(D): # 46, 47, 48つまりビーコンで言うと20, 21, 16の順
        d_linkid = ddata[d] 
        d_index = df_link[df_link['linkid'] == d_linkid].index
        d_index = d_index.tolist()
        d_index = d_index[0]

        # 価値関数
        z = Z[:, d].reshape(L, 1)
        z = z ** beta
        ZD = np.tile(z, (1, L))
        ZD = ZD.T

        # Mをdごとに更新
        M = np.zeros((L, L))
        for k in range(L): 
            for a in range(L): 
                Ika = I[k, a] 
                if Ika == 1:                            # 接続してなければ無関係（0のまま）
                    if a == d_index: 
                        M[k, a] = 1                     # 吸収リンクの即時効用はexp(0)=1（接続してたら）
                    else:
                        M[k, a] = Mset(x)[k, a]

        # 選択確率
        Mz = (M @ z != 0) * (M @ z) + (M @ z == 0)  
        MZ = np.tile(Mz, (1, L))  
        p = (M * ZD) / MZ

        #print(p) # パラメタ変えても
        print(f'pのlink2からとlink12からはそれぞれ{p[1]}{p[11]}')

        # 目的地がdのデータセット
        df = df_list[d]
        absorption = df.loc[0, 'absorption']
        grouped2 = df.groupby('userid')
        df_list2 = [group for name, group in grouped2] 
        df_list2 = [data2.reset_index(drop=True) for data2 in df_list2]
        count = 0

        for i in range(len(df_list2)): 
            dfi = df_list2[i] # 各自の結果（userid, timestep, maxbeacon, absorption）
            userid = dfi.loc[0, 'userid']
            LLi = 0 
            beacon_list = dfi['maxbeacon'].tolist() # 各時刻でscoreがmaxのビーコンが入ってる
            tsi = len(dfi)

            ###### tsnwの用意
            T = tsi # 時刻6は吸収リンクに行く時刻．それまでの制約を考えるなら時刻5までで良いか？→Tは全部の移動の時間制約

            # 起点Oからの到達可能性指示変数ioの定義
            Itlist = np.zeros((T+1, L, L))
            II = np.copy(I)
            Itlist[0, :, :] = np.eye(L) # ItlistにはT+1成分あるが，その第一成分(t=0)は単位行列ということ．t=1以降はfor内で計算

            for ts in range(1, T+1):
                Itlist[ts, :, :] = II # 時刻1では普通の接続行列
                II = np.dot(II, I) # 接続条件の積．つまり2回先に接続していたら1，そうでなければ0
                II = np.where(II > 0, 1, 0) 
                # この時点ではItlistには時刻0にスタートして時刻tsに利用可能なリンクが1になっているような行列が各時刻毎に入っている

            # 終点Dからの到達可能性指示変数idの定義
            Ittlist = np.zeros((T+1, L, L))
            for ts in range(T+1):
                Ittlist[ts, :, :] = np.transpose(Itlist[T - ts, :, :]) 

            saikyo_key = [1, 2, 3, 4, 5, 6]
            yamate_key = [7, 8, 9, 10, 11, 12, 13]
            discriminate_key = [19, 22, 23]

            ##### 時刻1はここで決めておく
            first_beacon = int(dfi.loc[0, 'maxbeacon']) 
            first_dist_list = d_array[:, first_beacon-1] 
            first_dist_dict = {index + 1: value for index, value in enumerate(first_dist_list)} # keyはlinkidになっている．max_beaconと各リンクの距離が入ってる

            if any(item in discriminate_key for item in beacon_list): # 埼京オンリーにする
                filtered_dict = {key: first_dist_dict[key] for key in first_dist_dict if key in saikyo_key}
                max_key = min(filtered_dict, key = filtered_dict.get) # これが初期リンクとなる．埼京ホームのうち，最もmax_beaconに近いリンク
            else: # 山手オンリー
                filtered_dict = {key: first_dist_dict[key] for key in first_dist_dict if key in yamate_key}
                max_key = min(filtered_dict, key = filtered_dict.get)
                    
            a0 = int(max_key) 
            at_cand_list = [a0] # at_cand_listの初期値としてa0を与える

            # ODのうちOとしてa0が確定できたので次にDを確定させにいく
            last_beacon = int(dfi.loc[tsi-1, 'maxbeacon']) # 一番下の行を指定
            last_link = 0
            if last_beacon == 16: # node39
                last_link = 25 # 最後のリンク
            elif last_beacon == 20: # node28
                last_link = 17
            elif last_beacon == 21: # node29
                last_link = 18      
            else:
                continue      

            # 以上でO=a0, D=Dが決まったのでこの個人iに対してId（時空間接続行列）を与える（IdはOD決まらないと定義できないので）
            Id = np.zeros((T, L, L))
            ao = a0-1
            ad = last_link-1
            for ts in range(T):
                if ts == 0:
                    Id[ts, ao, :] = I[ao, :] # 最初はoの接続行
                    continue
                        
                alist = np.where(Itlist[ts + 1, ao, :] == Ittlist[ts + 1, ad, :])[0]  # +1しない方がいいかも

                for a in alist:
                    if Itlist[ts + 1, ao, a] == Ittlist[ts + 1, ad, a]:
                        klist = np.where(I[:, a] == 1)[0]

                        for k in klist:
                            if len(np.where(Id[ts - 1, :, k] == 1)[0]) != 0:
                                Id[ts, k, a] = 1
            
            ###### at_1は確定される
            # at_1 = # リンク尤度がマックスのやつをとる
            # at_1の初期値はa0
            at_1 = a0 # a0はリンクid

            # 各時刻ごとにmaxbeaconを参照．dfiの長さが滞在timestep数と同じになる
            for t in range(1, len(dfi)): # t=1から見る                  
                max_beacon = int(dfi.loc[t, 'maxbeacon']) 
                ts = dfi.loc[t, 'timestep']
                
                # 時刻tでのmax_beaconと各リンクの距離
                dist_list = d_array[:, max_beacon-1] 
                dist_dict = {index + 1: value for index, value in enumerate(dist_list)} # keyはlinkidになっている．max_beaconと各リンクの距離が入ってる

                at_1_list = at_cand_list # t=1のときはa0が入ってるがat_1_listに渡して空にするのでOK
                at_cand_list = []    



                if t >= 1 and t <= (len(dfi)-2):  

                    # 毎回dictは初期化する
                    res_dict = {}  

                    ###### at_1は確定される
                    # リンク尤度がマックスのやつをとる

                    # ここに時空間制約を入れてat_candを大幅に絞りたい．とにかくat_candをいくら絞れるか．at_candが次のat_1_listになるから，ここ絞れると後も絞られる
                    Idt = Id[t, :, :]

                    # 時間制約はtsi = len(dfi)
                    a_row = Idt[at_1-1, :]

                    at_cand = np.where(a_row == 1)[0]+1 # at_1のプリズム内のリンク
                    # ひとまずat_candにはt-1にat_1だった時，tにおいて利用可能なリンクが入っている．これはat_1の接続条件からさらに絞られていることになる
                    
                    #print(f'結果として時刻{ts-1}のリンクは{at_1}になった．今は時刻{ts}で次の候補は{at_cand}になる')
                    if len(at_cand) == 0:
                        break
                    
                    at_cand_list.extend(list(set(at_cand) - set(at_cand_list))) # 重複なし．これが全atの候補(linkid)
                    
                    for at in at_cand: ####### at_cand！！！！にする
                        
                        dist = d_array[at-1, max_beacon-1]  
                            
                        if dist >= 300: # max_beaconとの距離が30m以上離れているとpmat=0になるので，そのようなリンクは排除
                            print('yes')
                            continue              

                        min_dist = d_array[:, max_beacon-1].min() # max_beaconに一番近いリンクとの距離
                        pmat = mq(dist, min_dist)
                        
                        nume = pmat * p[at_1-1, at-1]

                        # deno計算．at_1に接続しているリンクたち．                            
                        deno = 0
                        at_1_row = I[at_1-1, :]
                        at_prime_list = []
                        at_prime_list = np.where(at_1_row == 1)[0]+1 

                        for at_prime in at_prime_list:
                            dist = d_array[at_prime-1, max_beacon-1]
                            pmat_prime = mq(dist, min_dist)
                            deno += pmat_prime * p[at_1-1, at_prime-1]
                        if deno == 0:
                            continue

                        delta = nume / deno 
                        pp = (p[at_1-1, at-1] == 0) + ((p[at_1-1, at-1] != 0) * p[at_1-1, at-1])

                        # atにリンク尤度を入れる．これを辞書にする．valueが最大のものを次のat_1にする
                        # res_dict[at] = delta * np.log(pp) # 観測尤度を入れるのでdeltaだけ入る
                        res_dict[at] = (delta, delta * np.log(pp))

                        #print(f'今見てるんはuser{userid}時刻{ts}max_beacon{max_beacon}，{at_1}から{at}への遷移で，リンク尤度deltaは{delta}')
                        #print(f'今の時点で')

                    # delta*np.log(pp)の最大値をLLiに入れる．かつat_1を決める
                    # max_value_key = max(res_dict, key=res_dict.get)
                    max_delta_key = max(res_dict, key=lambda k: res_dict[k][0]) # deltaが最も大きいリンクのリンクidということ


                    if not res_dict:
                        break

                    else:
                        
                        LLi += res_dict[max_delta_key][1] # 最大リンクのdelta*np.log(pp)
                        
                        at_1 = max_delta_key # これはlinkid．毎回更新される
                    

                elif t == (len(dfi) - 1): # 最後の滞在リンクは確定させる
                    aT = None
                    if max_beacon == 16: # node39
                        aT = 25
                    elif max_beacon == 20: # node28
                        aT = 17
                    elif max_beacon == 21: # node29
                        aT = 18

                    if aT is not None:
                        dist = d_array[aT-1, max_beacon-1]
                        min_dist = d_array[:, max_beacon-1].min()
                        pmat = mq(dist, min_dist) 

                        # at_1をそのまま与えれば良い
                            #for at_1 in at_1_list:
                                #at_prime_list = []

                        nume = pmat * p[at_1-1, aT-1]
                        if nume == 0:
                            continue

                        at_1_row = I[at_1-1, :]
                        at_prime_list = np.where(at_1_row == 1)[0]+1

                        deno = 0
                        for at_prime in at_prime_list:
                            dist_prime = d_array[at_prime-1, max_beacon-1] 
                            pmat_prime = mq(dist_prime, min_dist) 
                            deno += pmat_prime * p[at_1-1, at_prime-1]

                        if deno == 0:
                            continue
                            
                        delta = nume / deno 
                        pp = (p[at_1-1, aT-1] == 0) + ((p[at_1-1, aT-1] != 0) * p[at_1-1, aT-1])
                        LLi += delta * np.log(pp) 
                        # print(f'user{userid}時刻{ts}つまり最後のmaxビーコンのidは{max_beacon}，リンクは{aT}デルタは{nume}/{deno}で{delta}，pmatは{pmat}，経路選択確率は{pp}')

            LL += LLi # 個人に対して和をとる
            count += 1

        print(f'パラメタ{x}のもとでLLが回り，計算したuserは{count}人でLL={LL}') # LLは負のはず
    
    return -LL

In [43]:
x = np.array((-1, 1))

print(loglikelihood(x)) # 経路長-10の時，pの各行の確率が0になっているわ

パラメタ[-1  1]で観測モデルを回します
paramaterは[-1  1]
pのlink2からとlink12からはそれぞれ[5.97181582e-10 1.18047522e-09 3.74159480e-09 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 9.99999994e-01 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 9.16604289e-10
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00][0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 5.55517752e-10 6.94397190e-10
 6.38845415e-10 0.000

KeyboardInterrupt: 

In [ ]:
x = np.array((-1, 1))
print(loglikelihood(x))

パラメタ[-1.   0.7]で観測モデルを回します
paramaterは[-1.   0.7]
[[0.00121413 0.00185994 0.         ... 0.         0.         0.        ]
 [0.00120912 0.00185227 0.00265261 ... 0.         0.         0.        ]
 [0.         0.00182693 0.00261632 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


KeyboardInterrupt: 

In [29]:
x = np.array((0, 0)) # -0.7, 0.8の時1058，00の時3811
print(loglikelihood(x))

パラメタ[0 0]で観測モデルを回します
paramaterは[0 0]
結果として時刻21.0のリンクは2になった．今は時刻22.0で次の候補は[ 1  2  3 26]になる
今見てるんはuser0.0時刻22.0max_beacon22，2から1への遷移で，リンク尤度deltaは0.15934713255256477
今見てるんはuser0.0時刻22.0max_beacon22，2から2への遷移で，リンク尤度deltaは0.18891889005249168
今見てるんはuser0.0時刻22.0max_beacon22，2から3への遷移で，リンク尤度deltaは0.05848482448000811
今見てるんはuser0.0時刻22.0max_beacon22，2から26への遷移で，リンク尤度deltaは0.46910693072935833
結果として時刻22.0のリンクは26になった．今は時刻23.0で次の候補は[27]になる
今見てるんはuser0.0時刻23.0max_beacon22，26から27への遷移で，リンク尤度deltaは0.2626410545138136
結果として時刻23.0のリンクは27になった．今は時刻24.0で次の候補は[24 25 29]になる
今見てるんはuser0.0時刻24.0max_beacon15，27から24への遷移で，リンク尤度deltaは0.08817120586084404
今見てるんはuser0.0時刻24.0max_beacon15，27から25への遷移で，リンク尤度deltaは0.3532814191091851
今見てるんはuser0.0時刻24.0max_beacon15，27から29への遷移で，リンク尤度deltaは0.5017882569772284
結果として時刻5.0のリンクは9になった．今は時刻6.0で次の候補は[ 8  9 30]になる
今見てるんはuser1.0時刻6.0max_beacon2，9から8への遷移で，リンク尤度deltaは0.07570928037071407
今見てるんはuser1.0時刻6.0max_beacon2，9から9への遷移で，リンク尤度deltaは0.3824498992479805
今見てるんはuser1.0時刻6.0max_beacon2，9から30

In [27]:
x = np.array((-0.7, 0.8))
print(loglikelihood(x))

パラメタ[-0.7  0.8]で観測モデルを回します
paramaterは[-0.7  0.8]
結果として時刻21.0のリンクは2になった．今は時刻22.0で次の候補は[ 1  2  3 26]になる
今見てるんはuser0.0時刻22.0max_beacon22，2から1への遷移で，リンク尤度deltaは0.00047980272874230113
今見てるんはuser0.0時刻22.0max_beacon22，2から2への遷移で，リンク尤度deltaは0.0009426577423420894
今見てるんはuser0.0時刻22.0max_beacon22，2から3への遷移で，リンク尤度deltaは0.0004149695431823786
今見てるんはuser0.0時刻22.0max_beacon22，2から26への遷移で，リンク尤度deltaは0.9976766891043534
結果として時刻22.0のリンクは26になった．今は時刻23.0で次の候補は[27]になる
今見てるんはuser0.0時刻23.0max_beacon22，26から27への遷移で，リンク尤度deltaは0.9999999262607887
結果として時刻23.0のリンクは27になった．今は時刻24.0で次の候補は[24 25 29]になる
今見てるんはuser0.0時刻24.0max_beacon15，27から24への遷移で，リンク尤度deltaは2.075312488074872e-07
今見てるんはuser0.0時刻24.0max_beacon15，27から25への遷移で，リンク尤度deltaは0.9999986113934283
今見てるんはuser0.0時刻24.0max_beacon15，27から29への遷移で，リンク尤度deltaは1.1810734958203187e-06
結果として時刻5.0のリンクは9になった．今は時刻6.0で次の候補は[ 8  9 30]になる
今見てるんはuser1.0時刻6.0max_beacon2，9から8への遷移で，リンク尤度deltaは0.0884760804585935
今見てるんはuser1.0時刻6.0max_beacon2，9から9への遷移で，リンク尤度deltaは0.4665748059083553
今見てるんはuser1.

KeyboardInterrupt: 

In [19]:
# 例としてres_dictが次のように定義されていると仮定します
res_dict = {
    'key1': (3.5, 5.2),
    'key2': (2.8, 4.9),
    'key3': (4.1, 6.3)
}

# 最も大きなdeltaの値を持つキーを取得
max_delta_key = max(res_dict, key=lambda k: res_dict[k][0])

# 対応するdelta * np.log(pp)を取得
result = res_dict[max_delta_key][1]
resdelta = res_dict[max_delta_key][0]

print(f"最も大きなdeltaの値を持つキー: {max_delta_key}")
print(f'deltaが最も大きいやつのdelta{resdelta}')
print(f"そのdelta * np.log(pp)の値: {result}") # 6.3のはず


最も大きなdeltaの値を持つキー: key3
deltaが最も大きいやつのdelta4.1
そのdelta * np.log(pp)の値: 6.3
